In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('../data/raw/analyst_ratings.csv')
print(df.shape)
print(list(df.columns))

(1407328, 6)
['Unnamed: 0', 'headline', 'url', 'publisher', 'date', 'stock']
